# Hw 3: Nearest Neighbors
## Submit to Dropbox Hw3 by Oct 21 

### Part 1. 

Your programs should read files provided. In this format, each instance is described on a single line. The feature values are separated by
commas, and the last value on each line is the class label (for classification).  Lines starting with '%' are comments

- Your programs should  implement a k-nearest neighbor learner in a function according to the following guidelines:
  - Assume that for classification tasks, the class attribute is named 'class' and it is the last attribute listed among all the attributes.
  - Assume that all features will be numeric.
  - Use Euclidean distance to compute distances between instances.
  - Implement basic k-NN.
  - If there is a tie among multiple instances to be in the k-nearest neighbors, the tie should be broken in favor of those instances that come first in the data file.
  - If there is a tie in the class predicted by the k-nearest neighbors, then among the classes that have the same number of votes, the tie should be broken in favor of the class comes first in the data file.
- You should include a function myKNN and should accept three arguments as follows:
  - myKNN(traindata,testdata, k)
  - The myKNN function should use the training set and the given value of k to make classifications/predictions for every instance in the test set. This can be called from a main calling function.
  - The main program should  use  p-fold cross validation (set p =10) with just the training data to select the value of k (used by NN) to use for the test set by evaluating k1 k2 k3…. (set it to any values you like) and selecting the one that results in the minimal cross-validated error within the training set.
  - To measure error, you should use mean absolute error. The following link shows how to use cross validation with python, including generating indices for each fold.
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

- As output, your programs should print the value of k used for the test set on the first line, and then the predictions for the test-set instances.
- For each instance in the test set, your program should print one line of output with spaces separating the fields.
- For a classification task, each output line should list the predicted class label, and actual class label.
- This should be followed by a line listing the number of correctly classified test instances, and the total number of instances in the test set.
- This should be followed by a line listing the mean absolute error for the test instances, and the total number of instances in the test set.
- Copy and paste this output to the .docx file you will submit to canvas.

You should test your code on the following two data sets:
- yeast_train.txt
- yeast_test.txt

### Part 2.

For this part you will explore the effect of the k parameter on predictive accuracy.

- For the yeast data set, draw a plot showing how test-set accuracy varies as a function of k. Your plot should show accuracy for k = 1, 5, 10, 15, 20 after p-fold cross validation (where p=10).
- For the yeast data set, construct confusion matrices for the k = 1 and k = 15 test-set results (you don’t need to do cross validation for this). Show these confusion matrices and briefly discuss what the matrices tell you about the effect of k on the misclassifications. See how to create confusion matrices here.

http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/

The python code for confusion matrices can be found at 
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

Put these results in the .docx file (from both parts) and submit to dropbox.

In [26]:
import numpy as np
import pandas as pd
from numpy.linalg import inv
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
import scipy.stats as sp
from sklearn.model_selection import KFold

In [2]:
def normalize(Data):
    n,m = Data.shape
    print(n,m)
    avg = np.mean(Data, axis=0);
    for i in range(0,m - 1):
        temp = Data[:,i] - avg[i]
        s = np.std(Data[:, i])
        Data[:, i] = temp / s
    return Data

In [3]:
def myKNN(train,labeltrain,test,k):
    n1, m1 = train.shape
    n2, m2 = test.shape
    D = euclidean_distances(train,test)
    D = D.transpose()
    Ytest = np.zeros(n2)
    for i in range(0,n2):
        ind = np.argsort(D[i,:])
        ktoplabels = labeltrain[ind[0:k]]
        L = sp.mode(ktoplabels)
        Ytest[i] = L.mode.item()
    return Ytest

In [4]:
with open("./data/yeast_train.txt", "r") as yeast_in:
    yeast_train_df = pd.read_csv(yeast_in, header=None)

In [9]:
label_train = yeast_train_df[8].to_numpy()
train = yeast_train_df.drop(yeast_train_df.columns[8], axis=1)

In [14]:
with open("./data/yeast_test.txt", "r") as yeast_in:
    yeast_test_df = pd.read_csv(yeast_in, header=None)

In [15]:
label_test = yeast_test_df[8].to_numpy()
test = yeast_test_df.drop(yeast_test_df.columns[8], axis=1)

In [16]:
n1, m1 = train.shape
n2, m2 = test.shape

data = np.concatenate((train, test), axis=0)
data = normalize(data)

train = data[0:n1, :]
test = data[n1:n1+n2, :]

1484 8


In [33]:
train.shape

(1039, 8)

In [18]:
test

array([[ 0.07197441, -0.80667199,  0.69212358, ...,  6.50962832,
        -0.17109621,  0.22      ],
       [-0.72946507, -0.64522881,  0.80754232, ..., -0.0991314 ,
         0.52121948,  0.25      ],
       [ 0.58198136, -0.24162084,  0.46128609, ..., -0.0991314 ,
         0.17506163,  0.26      ],
       ...,
       [-0.87518135, -0.24162084, -0.3466451 , ..., -0.0991314 ,
        -1.03649083,  0.26      ],
       [ 0.07197441, -0.40306402,  1.38463603, ..., -0.0991314 ,
         1.55969302,  0.22      ],
       [ 1.23770457,  0.5655951 , -1.61625127, ..., -0.0991314 ,
         1.04045625,  0.22      ]])

In [35]:
k = 14
y_test = myKNN(train, label_train, test, k)

/tmp/ipykernel_29232/12476209.py:10: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  L = sp.mode(ktoplabels)


In [36]:
acc = ((y_test==label_test).astype('uint8')).sum()/n2
print(acc)

0.21348314606741572


In [37]:
# for homework
Kchoices=np.array([5, 10 , 15, 20])
for i in range(0, len(Kchoices)):
    K = Kchoices[i]
    kf = KFold(n_splits=10)
    kf.get_n_splits(train)

for train_index, test_index in kf.split(train):
    Ytest = myKNN(train[train_index,:], label_train[train_index], train[test_index,:], K) #return value is the prediction
    n = test_index.shape
    print(y_test == label_train[test_index])
    # acc = ((y_test == label_train[test_index],).astype('uint8')).sum()/n[0]
# AvgAcc across each fold for a particular choice of K
# bestK=find the K with maximum Avgacc

/tmp/ipykernel_29232/12476209.py:10: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  L = sp.mode(ktoplabels)


ValueError: operands could not be broadcast together with shapes (445,) (104,) 